In [ ]:
!pip install catboost -q
!pip install optuna -q

In [ ]:
import os
import sys
import pandas as pd
import numpy as np

# 모델
import sklearn 
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import f1_score
import catboost
from catboost import CatBoostClassifier
import optuna
from optuna.samplers import TPESampler
import joblib

np.random.seed(777)

In [ ]:
train = pd.read_csv('./data/train_data.csv')
test = pd.read_csv('./data/test_data.csv')
sample_submission = pd.read_csv("./data/sample_submission.csv")

In [ ]:
# 데이터 셋 나누기
X = train.drop('target', axis=1)
y = train['target']
X_test = test.copy()

In [ ]:
# Optuna로 탐색한 최적의 하이퍼 파라미터 load
study = joblib.load("./catboost_optuna_parameter.pkl") # 로드

In [ ]:
# K-fold 교차 검증을 통한 과적합 방지
is_holdout = False
cv = KFold(n_splits=5, shuffle=True, random_state=777)

scores = []
models = []

for tri, vai in cv.split(X):
    preds = []
    model = CatBoostClassifier(**study.best_params,
                               random_state = 777,
                               eval_metric="F1",
                               cat_features=X.drop(['contents_rn_cnt', 'person_rn_cnt'], axis=1).columns.tolist(),
                               one_hot_max_size=4)
    
    model.fit(X.iloc[tri], y[tri], eval_set=[(X.iloc[vai], y[vai])], 
              early_stopping_rounds=50, verbose = False)
    
    models.append(model)
    scores.append(model.get_best_score()["validation"]["F1"])

    if is_holdout:
        break

- 데이터에 범주형 데이터가 많기 때문에 Catboost를 사용해줌
- 모델 평가 방식이 F1 score이므로 F1 score를 높이는 방향으로 Optuna 진행
- 컨텐츠 빈도수 및 사용자 빈도수를 제외한 변수를 cat_feature 파라미터에 설정
- K-fold Cross Validation의 경우 K를 5로 설정함

In [ ]:
# 임계값을 0.4로 설정해 재현율을 높임
threshold = 0.4

pred_list = []
scores = []

for i,(tri, vai) in enumerate( cv.split(X) ):
    pred = models[i].predict_proba(X.iloc[vai])[:, 1]
    pred = np.where(pred >= threshold , 1, 0)
    score = f1_score(y[vai],pred)
    scores.append(score)
    pred = models[i].predict_proba(X_test)[:, 1]
    pred_list.append(pred)

print(scores)
print(np.mean(scores)) # 5번 검증된 결과의 평균

pred = np.mean( pred_list , axis = 0 )
pred = np.where(pred >= threshold , 1, 0)

[0.7195355906021874, 0.7181923714759536, 0.7144940215806358, 0.7169272895223014, 0.7149698415854709]
0.7168238229533098


In [ ]:
# 제출 데이터 생성 및 저장
sample_submission['target'] = pred
sample_submission.to_csv('./data/result_submission.csv',index=False)